# Quick test of Excel in Python

In [68]:
%reset -f

In [35]:
from xlcalculator import ModelCompiler
from xlcalculator import Model
from xlcalculator import Evaluator

import pandas as pd

filename = r'basic.xlsx'
json_file_name = filename + r'.json'


Display the Excel file, the first row shows the formulas of the cells below.

In [36]:
pd.read_excel(filename)

,X,2X,X*X,C2,C2*2
0,1,2,1,1,2


# Turn the excel file into a python usable code

In [37]:
compiler = ModelCompiler()
new_model = compiler.read_and_parse_archive(filename, build_code=False)
new_model.persist_to_json_file(json_file_name)

In [38]:
reconstituted_model = Model()
reconstituted_model.construct_from_json_file(json_file_name, build_code=True)

evaluator = Evaluator(reconstituted_model)


In [39]:
val = evaluator.evaluate('Sheet1!B2')
print("value 'evaluated' for Sheet1!B2 before update:", val)
val = evaluator.evaluate('Sheet1!C2')
print("value 'evaluated' for Sheet1!C2 before update:", val)
val = evaluator.evaluate('Sheet1!E2')
print("value 'evaluated' for Sheet1!E2 before update:", val)

value 'evaluated' for Sheet1!B2 before update: 2
value 'evaluated' for Sheet1!C2 before update: 1
value 'evaluated' for Sheet1!E2 before update: 2


# Test: Change a cell and re evaluate the other cells

Make A2=2

In [40]:
# Now we change the value of A2
evaluator.set_cell_value('Sheet1!A2', 2)

# Evaluate the needed cells
val = evaluator.evaluate('Sheet1!B2')
print("New value for Sheet1!B2:", val)
val = evaluator.evaluate('Sheet1!C2')
print("New value for Sheet1!C2:", val)
val = evaluator.evaluate('Sheet1!E2')
print("New value for Sheet1!E2:", val)

New value for Sheet1!B2: 4
New value for Sheet1!C2: 4
New value for Sheet1!E2: 8


Make A2=3

In [41]:
# Now we change the value of A2
evaluator.set_cell_value('Sheet1!A2', 3)

# Evaluate the needed cells
val = evaluator.evaluate('Sheet1!B2')
print("New value for Sheet1!B2:", val)
val = evaluator.evaluate('Sheet1!C2')
print("New value for Sheet1!C2:", val)
val = evaluator.evaluate('Sheet1!E2')
print("New value for Sheet1!E2:", val)

New value for Sheet1!B2: 6
New value for Sheet1!C2: 9
New value for Sheet1!E2: 18


In [44]:
print(evaluator.evaluate('Sheet1!A2:C2'))

# Now try loading the Excel simulator

In [14]:
filename = r'carlin_soskice_macroeconomic_simulator.xlsm'
json_file_name = filename + r'.json'

compiler = ModelCompiler()
new_model = compiler.read_and_parse_archive(filename, build_code=False)
new_model.persist_to_json_file(json_file_name)

ValueError: B4=1 is not a valid coordinate or range

Something went wrong, but it might be fixable. **I don't know**. 
